In [1]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import sys
import subprocess
import pdb
import time
import datetime
import math
import random
import _pickle as cPickle
from collections import defaultdict

from six.moves import zip_longest
import numpy as np

import tensorflow as tf
from tensorflow import distributions as tfd
from tensorflow.keras.preprocessing.sequence import pad_sequences

from data_structure import get_batches
from components import tf_log, sample_latents, compute_kl_loss, dynamic_rnn, dynamic_bi_rnn
from topic_model import TopicModel

from topic_beam_search_decoder import BeamSearchDecoder

In [2]:
PAD = '<pad>' # This has a vocab id, which is used to pad the encoder input, decoder input and target sequence
UNK = '<unk>' # This has a vocab id, which is used to represent out-of-vocabulary words
BOS = '<p>' # This has a vocab id, which is used at the beginning of every decoder input sequence
EOS = '</p>' # This has a vocab id, which is used at the end of untruncated target sequences

# load data & set config

In [3]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)

flags = tf.app.flags

flags.DEFINE_string('gpu', '3', 'visible gpu')

flags.DEFINE_string('mode', 'train', 'set train or eval')

flags.DEFINE_string('data_path', 'data/apnews/instances.pkl', 'path of data')
flags.DEFINE_string('modeldir', 'model/tglm_vae_tmp', 'directory of model')
flags.DEFINE_string('modelname', 'apnews', 'name of model')

flags.DEFINE_integer('epochs', 50, 'epochs')
flags.DEFINE_integer('batch_size', 64, 'number of sentences in each batch')
flags.DEFINE_integer('log_period', 500, 'valid period')

flags.DEFINE_string('opt', 'Adagrad', 'optimizer')
flags.DEFINE_float('lr', 0.1, 'lr')
flags.DEFINE_float('reg', 1., 'regularization term')
flags.DEFINE_float('grad_clip', 5., 'grad_clip')

flags.DEFINE_float('keep_prob', 0.8, 'dropout rate')
flags.DEFINE_float('word_keep_prob', 0.75, 'word dropout rate')

flags.DEFINE_bool('warmup', True, 'flg of warming up')
flags.DEFINE_integer('epochs_cycle', 10, 'number of epochs within a cycle')
flags.DEFINE_float('r_cycle', 0.5, 'proportion used to increase beta within a cycle')
flags.DEFINE_integer('warmup_topic', 0, 'warmup period for KL of topic')

flags.DEFINE_integer('beam_width', 2, 'beam_width')
flags.DEFINE_float('length_penalty_weight', 0.0, 'length_penalty_weight')

flags.DEFINE_integer('n_topic', 10, 'number of topic')
flags.DEFINE_integer('dim_hidden_bow', 256, 'dim of hidden bow')
flags.DEFINE_integer('dim_latent_bow', 32, 'dim of latent topic')
flags.DEFINE_integer('dim_emb', 256, 'dim_emb')
flags.DEFINE_integer('dim_hidden', 512, 'dim_hidden')
flags.DEFINE_integer('dim_hidden_topic', 512, 'dim_hidden_topic')
flags.DEFINE_integer('dim_latent', 32, 'dim_latent')
flags.DEFINE_bool('bidirectional', True, 'flg of bidirectional encoding')

# for evaluation
flags.DEFINE_string('refdir', 'ref', 'refdir')
flags.DEFINE_string('outdir', 'out', 'outdir')

flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_bool('logtostderr', True, 'kernel')
flags.DEFINE_bool('showprefixforinfo', False, '')
flags.DEFINE_bool('verbosity', False, '')
# flags.DEFINE_integer('stderrthreshold', 20, 'kernel')

config = flags.FLAGS

flags.DEFINE_string('modelpath', os.path.join(config.modeldir, config.modelname), 'path of model')

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.data_path,'rb'))

In [6]:
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)

In [7]:
flags.DEFINE_integer('PAD_IDX', word_to_idx[PAD], 'PAD_IDX')
flags.DEFINE_integer('UNK_IDX', word_to_idx[UNK], 'UNK_IDX')
flags.DEFINE_integer('BOS_IDX', word_to_idx[BOS], 'BOS_IDX')
flags.DEFINE_integer('EOS_IDX', word_to_idx[EOS], 'EOS_IDX')

flags.DEFINE_integer('n_vocab', len(word_to_idx), 'n_vocab')
flags.DEFINE_integer('dim_bow', len(bow_idxs), 'dim_bow')

maximum_iterations = max([max([instance.max_sent_l for instance in batch]) for ct, batch in dev_batches])
flags.DEFINE_integer('maximum_iterations', maximum_iterations, 'maximum_iterations')

flags.DEFINE_integer('cycle_steps', len(train_batches)*config.epochs_cycle, 'number of steps for each cycle')

# build language model 

## feed dict

In [8]:
tf.reset_default_graph()

t_variables = {}
t_variables['bow'] = tf.placeholder(tf.float32, [None, config.dim_bow], name='bow')
t_variables['input_token_idxs'] = tf.placeholder(tf.int32, [None, None], name='input_token_idxs')
t_variables['dec_input_idxs'] = tf.placeholder(tf.int32, [None, None], name='dec_input_idxs')
t_variables['dec_target_idxs'] = tf.placeholder(tf.int32, [None, None], name='dec_target_idxs')
t_variables['batch_l'] = tf.placeholder(tf.int32, name='batch_l')
t_variables['doc_l'] = tf.placeholder(tf.int32, [None], name='doc_l')
t_variables['sent_l'] = tf.placeholder(tf.int32, [None], name='sent_l')
t_variables['keep_prob'] = tf.placeholder(tf.float32, name='keep_prob')

In [9]:
def get_feed_dict(batch, mode='train', assertion=False):
    def token_dropout(sent_idxs):
        sent_idxs_dropout = np.asarray(sent_idxs)
        sent_idxs_dropout[np.random.rand(len(sent_idxs)) > config.word_keep_prob] = config.UNK_IDX
        return list(sent_idxs_dropout)

    bow = np.array([instance.bow for instance in batch]).astype(np.float32)
    
    doc_l = np.array([len(instance.token_idxs) for instance in batch])
    
    feed_input_token_idxs_list = [sent_idxs for instance in batch for sent_idxs in instance.token_idxs]
    feed_dec_input_idxs_list = [[config.BOS_IDX] + token_dropout(sent_idxs) for sent_idxs in feed_input_token_idxs_list]
    feed_dec_target_idxs_list = [sent_idxs + [config.EOS_IDX]  for sent_idxs in feed_input_token_idxs_list]
        
    sent_l = np.array([len(sent_idxs) for sent_idxs in feed_input_token_idxs_list], np.int32)
    batch_l = len(sent_l)
    
    feed_input_token_idxs = pad_sequences(feed_input_token_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_input_idxs = pad_sequences(feed_dec_input_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    feed_dec_target_idxs = pad_sequences(feed_dec_target_idxs_list, padding='post', value=config.PAD_IDX, dtype=np.int32)
    
    if assertion:
        index = 0
        for instance in batch:
            for line_idxs in instance.token_idxs:
                assert feed_input_token_idxs_list[index] == line_idxs
                index += 1
        assert feed_input_token_idxs.shape[1] == np.max(sent_l)
        assert feed_dec_input_idxs.shape[1] == np.max(sent_l) + 1
        assert feed_dec_target_idxs.shape[1] == np.max(sent_l) + 1
    
    keep_prob = config.keep_prob if mode == 'train' else 1.0

    feed_dict = {
                t_variables['bow']: bow, 
                t_variables['batch_l']: batch_l, t_variables['doc_l']: doc_l, t_variables['sent_l']: sent_l, 
                t_variables['input_token_idxs']: feed_input_token_idxs, t_variables['dec_input_idxs']: feed_dec_input_idxs, t_variables['dec_target_idxs']: feed_dec_target_idxs, 
                t_variables['keep_prob']: keep_prob
    }
    return  feed_dict

In [10]:
def debug_shape(variables):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)

def debug_value(variables, return_value=False):
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)

    if return_value: 
        return _variables
    else:
        for _variable, variable in zip(_variables, variables):
            if hasattr(variable, 'name'):
                print(variable.name, ':', _variable)
            else:
                print(_variable)
                
def check_shape(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable.shape)
        else:
            print(_variable.shape)
            
    sess.close()
    
def check_value(variables):
    if 'sess' in globals(): raise
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    
    sample_batch = test_batches[0][1]
    feed_dict = get_feed_dict(sample_batch)
    _variables = sess.run(variables, feed_dict=feed_dict)
    for _variable, variable in zip(_variables, variables):
        if hasattr(variable, 'name'):
            print(variable.name, ':', _variable)
        else:
            print(_variable.shape)
            
    sess.close()    
    
# sent_loss_kl_categ_tmp = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, tf_log(prob_topic_infer/prob_topic_sents)), 1))
# debug_value([sent_loss_kl_categ, sent_loss_kl_categ_tmp])
# sent_loss_kl_gauss_tmp = 0.5 * tf.reduce_sum(tf.exp(logvars_topic_infer-logvars_topic) + tf.square(means_topic - means_topic_infer) / tf.exp(logvars_topic) - 1 + (logvars_topic - logvars_topic_infer), -1)
# sent_loss_kl_gmm_tmp = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, sent_loss_kl_gauss_tmp), -1))
# debug_value([sent_loss_kl_gmm_tmp, sent_loss_kl_gmm])    

## topic model

In [11]:
# encode bow
with tf.variable_scope('topic/enc', reuse=False):
    hidden_bow_ = tf.layers.Dense(units=config.dim_hidden_bow, activation=tf.nn.relu, name='hidden_bow')(t_variables['bow'])
    hidden_bow = tf.layers.Dropout(t_variables['keep_prob'])(hidden_bow_)
    means_bow = tf.layers.Dense(units=config.dim_latent_bow, name='mean_bow')(hidden_bow)
    logvars_bow = tf.layers.Dense(units=config.dim_latent_bow, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_bow)
    latents_bow = sample_latents(means_bow, logvars_bow) # sample latent vectors

    prob_topic = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic')(latents_bow) # inference of topic probabilities

# decode bow
with tf.variable_scope('shared', reuse=False):
    embeddings = tf.get_variable('emb', [config.n_vocab, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of vocab

bow_embeddings = tf.nn.embedding_lookup(embeddings, bow_idxs) # embeddings of each bow features

with tf.variable_scope('topic/dec', reuse=False):
    topic_embeddings = tf.get_variable('topic_emb', [config.n_topic, config.dim_emb], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer()) # embeddings of topics

    topic_bow = tf.nn.softmax(tf.matmul(topic_embeddings, bow_embeddings, transpose_b=True), 1) # bow vectors for each topic
    logits_bow = tf_log(tf.matmul(prob_topic, topic_bow)) # predicted bow distribution

    # prior of each gaussian distribution (computed for each topic)
    hidden_topic = tf.layers.Dense(units=config.dim_hidden_topic, activation=tf.nn.relu, name='hidden_topic')(topic_bow)
    means_topic = tf.layers.Dense(units=config.dim_latent, name='mean_topic')(hidden_topic)
    logvars_topic = tf.layers.Dense(units=config.dim_latent, kernel_initializer=tf.constant_initializer(0), bias_initializer=tf.constant_initializer(0), name='logvar_topic')(hidden_topic)
    sigma_topic = tf.exp(0.5 * logvars_topic)
    gauss_topic = tfd.Normal(loc=means_topic, scale=sigma_topic)    
    
# define losses
topic_losses_recon = -tf.reduce_sum(tf.multiply(t_variables['bow'], logits_bow), 1)
topic_loss_recon = tf.reduce_mean(topic_losses_recon) # negative log likelihood of each words

topic_loss_kl = compute_kl_loss(means_bow, logvars_bow) # KL divergence b/w latent dist & gaussian std

topic_bow_norm = topic_bow / tf.norm(topic_bow, axis=1, keepdims=True)
topic_dots = tf.clip_by_value(tf.matmul(topic_bow_norm, tf.transpose(topic_bow_norm)), -1., 1.)
topic_loss_reg = tf.reduce_mean(tf.square(topic_dots - tf.eye(config.n_topic)))
# topic_angles = tf.acos(topic_dots)
# topic_angles_mean = tf.reduce_mean(topic_angles)
# topic_angles_vars = tf.reduce_mean(tf.square(topic_angles - topic_angles_mean))
# topic_loss_reg = tf.exp(topic_angles_vars - topic_angles_mean)

# monitor
n_bow = tf.reduce_sum(t_variables['bow'], 1)
topic_ppls = tf.divide(topic_losses_recon, n_bow)
topics_freq_bow_indices = tf.nn.top_k(topic_bow, 10, name='topic_freq_bow').indices

## encoder

In [12]:
# input
input_token_idxs = t_variables['input_token_idxs']
batch_l = t_variables['batch_l']
sent_l = t_variables['sent_l']
max_sent_l = tf.reduce_max(sent_l)

with tf.variable_scope('sent/enc', reuse=False):
    # get word embedding
    enc_input = tf.nn.embedding_lookup(embeddings, input_token_idxs)

    # get sentence embedding
    _, enc_state = dynamic_bi_rnn(enc_input, sent_l, config.dim_hidden, t_variables['keep_prob'])

    # TODO House Holder flow
    hidden_topic_infer =  tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='hidden_topic_infer')(enc_state)
    prob_topic_infer = tf.layers.Dense(units=config.n_topic, activation=tf.nn.softmax, name='prob_topic_infer')(hidden_topic_infer)

    w_mean_topic_infer = tf.get_variable('mean_topic_infer/kernel', [config.n_topic, enc_state.shape[-1], config.dim_latent], dtype=tf.float32)
    b_mean_topic_infer = tf.get_variable('mean_topic_infer/bias', [1, config.n_topic, config.dim_latent], dtype=tf.float32)
    means_topic_infer = tf.tensordot(enc_state, w_mean_topic_infer, axes=[[1], [1]]) + b_mean_topic_infer
    
    w_logvar_topic_infer = tf.get_variable('logvar_topic_infer/kernel', [config.n_topic, enc_state.shape[-1], config.dim_latent], dtype=tf.float32, initializer=tf.constant_initializer(0))
    b_logvar_topic_infer = tf.get_variable('logvar_topic_infer/bias', [1, config.n_topic, config.dim_latent], dtype=tf.float32, initializer=tf.constant_initializer(0))
    logvars_topic_infer = tf.tensordot(enc_state, w_logvar_topic_infer, axes=[[1], [1]]) + b_logvar_topic_infer
    sigma_topic_infer = tf.exp(0.5 * logvars_topic_infer)
    gauss_topic_infer = tfd.Normal(loc=means_topic_infer, scale=sigma_topic_infer)
    
    # latent vectors from each gaussian dist.
    latents_topic_infer = sample_latents(means_topic_infer, logvars_topic_infer) 
    # latent vector from gaussian mixture
    latents_input = tf.matmul(tf.expand_dims(prob_topic_infer, -1), latents_topic_infer, transpose_a=True)
    
    # for beam search
    means_input = tf.matmul(tf.expand_dims(prob_topic_infer, -1), means_topic_infer, transpose_a=True)    

## decoder

In [13]:
# prepare for decoding
dec_sent_l = tf.add(sent_l, 1)
dec_input_idxs = t_variables['dec_input_idxs']
dec_input = tf.nn.embedding_lookup(embeddings, dec_input_idxs)

dec_latents_input = tf.tile(latents_input, [1, tf.shape(dec_input)[1], 1])
dec_concat_input = tf.concat([dec_input, dec_latents_input], -1)

# decode for training
with tf.variable_scope('sent/dec/rnn', initializer=tf.contrib.layers.xavier_initializer(), dtype = tf.float32, reuse=False):
    dec_cell = tf.contrib.rnn.GRUCell(config.dim_hidden)
    dec_cell = tf.contrib.rnn.DropoutWrapper(dec_cell, output_keep_prob = t_variables['keep_prob'])

    dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(tf.squeeze(latents_input, 1))
    
    helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_concat_input, sequence_length=dec_sent_l)

    train_decoder = tf.contrib.seq2seq.BasicDecoder(
        cell=dec_cell,
        helper=helper,
        initial_state=dec_initial_state)

    dec_outputs, _, output_sent_l = tf.contrib.seq2seq.dynamic_decode(train_decoder)
    
    output_layer = tf.layers.Dense(config.n_vocab, use_bias=False, name='out')
    output_logits = output_layer(dec_outputs.rnn_output)
    
    output_token_idxs = tf.argmax(output_logits, 2)

In [14]:
start_tokens = tf.fill([batch_l], config.BOS_IDX)
end_token = config.EOS_IDX

with tf.variable_scope('sent/dec/rnn', reuse=True):
    infer_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(tf.squeeze(means_input, 1))
    beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(infer_dec_initial_state, multiplier=config.beam_width)
    beam_latents_input = tf.contrib.seq2seq.tile_batch(tf.squeeze(means_input, 1), multiplier=config.beam_width) # added
    
    beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=beam_latents_input)

    beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        beam_decoder,
        maximum_iterations = config.maximum_iterations)

    beam_output_token_idxs = beam_dec_outputs.predicted_ids[:, :, 0]

In [15]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    inter_means_input = tf.placeholder(tf.float32, [None, config.dim_latent])
    
    inter_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(inter_means_input)
    inter_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(inter_dec_initial_state, multiplier=config.beam_width)
    inter_beam_latents_input = tf.contrib.seq2seq.tile_batch(inter_means_input, multiplier=config.beam_width) # added
    
    inter_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=inter_beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=inter_beam_latents_input)

    inter_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        inter_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    inter_beam_output_token_idxs = inter_beam_dec_outputs.predicted_ids[:, :, 0]

In [16]:
with tf.variable_scope('sent/dec/rnn', reuse=True):
    topic_dec_initial_state = tf.layers.Dense(units=config.dim_hidden, activation=tf.nn.relu, name='init_state')(means_topic)
    topic_beam_dec_initial_state = tf.contrib.seq2seq.tile_batch(topic_dec_initial_state, multiplier=config.beam_width)
    topic_beam_latents_input = tf.contrib.seq2seq.tile_batch(means_topic, multiplier=config.beam_width) # added
    
    topic_beam_decoder = BeamSearchDecoder(
        cell=dec_cell,
        embedding=embeddings,
        start_tokens=start_tokens,
        end_token=end_token,
        initial_state=topic_beam_dec_initial_state,
        beam_width=config.beam_width, 
        output_layer=output_layer,
        length_penalty_weight=config.length_penalty_weight,
        latents_input=topic_beam_latents_input)

    topic_beam_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(
        topic_beam_decoder,
        maximum_iterations = config.maximum_iterations)

    topic_beam_output_token_idxs = topic_beam_dec_outputs.predicted_ids[:, :, 0]

## language modeling cost

In [17]:
# target and mask
dec_target_idxs = t_variables['dec_target_idxs']
dec_mask_tokens = tf.sequence_mask(dec_sent_l, maxlen=max_sent_l+1, dtype=tf.float32)

# nll for each token (averaged over batch & sentence)
sent_loss_recon = tf.contrib.seq2seq.sequence_loss(output_logits, dec_target_idxs, dec_mask_tokens)

In [18]:
doc_l = t_variables['doc_l']
mask_sents = tf.sequence_mask(doc_l)
mask_sents_flatten = tf.reshape(mask_sents, [tf.shape(mask_sents)[0]*tf.shape(mask_sents)[1]])

prob_topic_tiled = tf.tile(tf.expand_dims(prob_topic, 1), [1, tf.shape(mask_sents)[1], 1])
prob_topic_flatten = tf.reshape(prob_topic_tiled, [tf.shape(mask_sents)[0]*tf.shape(mask_sents)[1], config.n_topic])
prob_topic_sents = tf.boolean_mask(prob_topic_flatten, mask_sents_flatten)

In [19]:
# inferred mixture probabilities (computed for each sentence)
categ_topic_infer = tfd.Categorical(probs=prob_topic_infer)

# prior of mixture probabilities (computed for each document, tiled for each sentence)
categ_topic = tfd.Categorical(probs=prob_topic_sents)

sent_loss_kl_categ = tf.reduce_mean(tfd.kl_divergence(categ_topic_infer, categ_topic))

# inference of each gaussian gaussribution (computed for each sentence)

sent_loss_kl_gauss = tf.reduce_sum(tfd.kl_divergence(gauss_topic_infer, gauss_topic), -1)
sent_loss_kl_gmm = tf.reduce_mean(tf.reduce_sum(tf.multiply(prob_topic_infer, sent_loss_kl_gauss), -1))

sent_loss_kl = sent_loss_kl_categ + sent_loss_kl_gmm

## optimizer

In [20]:
global_step = tf.Variable(0, name='global_step',trainable=False)
tau = tf.cast(tf.divide(tf.mod(global_step, tf.constant(config.cycle_steps)), tf.constant(config.cycle_steps)), dtype=tf.float32)
beta = tf.minimum(1., tau/config.r_cycle)

sent_loss = sent_loss_recon + beta * sent_loss_kl

topic_loss = topic_loss_recon + topic_loss_kl + config.reg * topic_loss_reg
loss = topic_loss + sent_loss

# define optimizer
if config.opt == 'Adam':
    optimizer = tf.train.AdamOptimizer(config.lr)
elif config.opt == 'Adagrad':
    optimizer = tf.train.AdagradOptimizer(config.lr)
    
grad_vars = optimizer.compute_gradients(loss)
clipped_grad_vars = [(tf.clip_by_value(grad, -config.grad_clip, config.grad_clip), var) for grad, var in grad_vars]

opt = optimizer.apply_gradients(clipped_grad_vars, global_step=global_step)

/home/m-isonuma/.pyenv/versions/anaconda2-5.3.0/envs/py36/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


# run model 

In [21]:
def idxs_to_sents(token_idxs, config, idx_to_word):
    sents = []
    for sent_idxs in token_idxs:
        tokens = []
        for idx in sent_idxs:
            if idx == config.EOS_IDX: break
            tokens.append(idx_to_word[idx])
        sent = ' '.join(tokens)
        sents.append(sent)
    return sents

In [22]:
def get_loss(sess, batches):
    losses = []
    ppl_list = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_batch, sent_loss_batch, ppls_batch = sess.run([loss, topic_loss, sent_loss, topic_ppls], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_batch, sent_loss_batch]]
        ppl_list += list(ppls_batch)
    loss_mean, topic_loss_mean, sent_loss_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    return loss_mean, topic_loss_mean, sent_loss_mean, ppl_mean

def get_all_losses(sess, batches):
    losses = []
    for ct, batch in batches:
        feed_dict = get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, sent_loss_recon_batch, sent_loss_kl_batch = \
        sess.run([loss, topic_loss_recon, topic_loss_kl, sent_loss_recon, sent_loss_kl], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
    print('LOSS %.2f | TM NLL: %.2f, KL: %.4f | LM NLL: %.2f, KL: %.4f' %  np.mean(losses, 0))

In [23]:
def print_sample(sample_batch):
    feed_dict = get_feed_dict(sample_batch)
    pred_token_idxs = sess.run(output_token_idxs, feed_dict = feed_dict)
    true_token_idxs = [sent_idxs for instance in sample_batch for sent_idxs in instance.token_idxs]
    
    assert len(pred_token_idxs) == len(true_token_idxs)
    
    pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)
    true_sents = idxs_to_sents(true_token_idxs, config, idx_to_word)
    
    for i, (true_sent, pred_sent) in enumerate(zip(true_sents, pred_sents)):        
        print(i, 'True: %s' % true_sent)
        print(i, 'Pred: %s' % pred_sent)

def print_topic_sample():
    pred_topics_freq_bow_indices, pred_topic_token_idxs = sess.run([topics_freq_bow_indices, topic_beam_output_token_idxs], 
                                                                                                           feed_dict={t_variables['batch_l']: config.n_topic, t_variables['keep_prob']: 1.,})
    pred_topic_sents = idxs_to_sents(pred_topic_token_idxs, config, idx_to_word)
    
    topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]
    
    print('-----------Topic Samples-----------')
    for i, (topic_freq_bow_idxs, pred_topic_sent) in enumerate(zip(topics_freq_bow_idxs, pred_topic_sents)):
        print(i, ' bow:', ' '.join([idx_to_word[idx] for idx in topic_freq_bow_idxs]))
        print(i, ' sent:', pred_topic_sent)

In [24]:
if 'sess' in globals(): sess.close()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

logs = []
losses_train = []
ppls_train = []
loss_min = np.inf
beta_eval = 1.
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)
saver = tf.train.Saver(max_to_keep=10)

In [25]:
if len(logs) == 0:
    cmd_rm = 'rm -r %s' % config.modeldir
    res = subprocess.call(cmd_rm.split())

    cmd_mk = 'mkdir %s' % config.modeldir
    res = subprocess.call(cmd_mk.split())

time_start = time.time()
while epoch < config.epochs:
    for ct, batch in train_batches:
        feed_dict = get_feed_dict(batch)

        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch, sent_loss_kl_categ_batch, sent_loss_kl_gmm_batch, ppls_batch = \
        sess.run([opt, loss, topic_loss_recon, topic_loss_kl, topic_loss_reg, sent_loss_recon, sent_loss_kl, sent_loss_kl_categ, sent_loss_kl_gmm, topic_ppls], feed_dict = feed_dict)
   
        if sent_loss_kl_batch == np.inf:
            print('Nan occured')
            ckpt = tf.train.get_checkpoint_state(config.modeldir)
            model_checkpoint_path = ckpt.all_model_checkpoint_paths[-1]
            saver.restore(sess, model_checkpoint_path)            
            break
            
        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_reg_batch, sent_loss_recon_batch, sent_loss_kl_batch]]
        ppls_train += list(ppls_batch)

        if ct%config.log_period==0:
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train, sent_loss_recon_train, sent_loss_kl_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_dev, sent_loss_dev, ppl_dev = get_loss(sess, dev_batches)

            if config.warmup: beta_eval = beta.eval(session=sess)
            global_step_log = sess.run(tf.train.get_global_step())            
            
#             if loss_dev < loss_min:
#                 loss_min = loss_dev
#                 saver.save(sess, config.modelpath, global_step=global_step_log)

            clear_output()

            time_finish = time.time()
            time_log = int(time_finish - time_start)
            logs += [(time_log, epoch, ct, loss_train, ppl_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_reg_train, sent_loss_recon_train, sent_loss_kl_train, loss_dev, ppl_dev, topic_loss_dev, sent_loss_dev, beta_eval)]
            for log in logs:
                print('%03d[s], Ep: %02d, Ct: %05d|TR LOSS: %.0f, PPL: %.0f|TM NLL: %.0f, KL: %.2f, REG:%.2f | LM NLL: %.2f, KL: %.2f|DE LOSS: %.0f, PPL: %.0f, TM: %.0f, LM: %.2f|BETA: %.6f' %  log)

            print_sample(batch)

            time_start = time.time()
            
            print_topic_sample()
                
    epoch += 1
    train_batches = get_batches(instances_train, config.batch_size, iterator=True)

010[s], Ep: 00, Ct: 00000|TR LOSS: 364, PPL: 2660|TM NLL: 352, KL: 0.76, REG:0.90 | LM NLL: 10.34, KL: 1.60|DE LOSS: 348, PPL: 2657, TM: 337, LM: 10.34|BETA: 0.000069
082[s], Ep: 00, Ct: 00500|TR LOSS: 329, PPL: 1695|TM NLL: 320, KL: 0.51, REG:0.53 | LM NLL: 7.43, KL: 2.10|DE LOSS: 321, PPL: 1532, TM: 314, LM: 6.94|BETA: 0.034743
081[s], Ep: 00, Ct: 01000|TR LOSS: 326, PPL: 1555|TM NLL: 317, KL: 1.35, REG:0.35 | LM NLL: 7.15, KL: 2.05|DE LOSS: 317, PPL: 1353, TM: 310, LM: 6.91|BETA: 0.069417
082[s], Ep: 00, Ct: 01500|TR LOSS: 324, PPL: 1474|TM NLL: 315, KL: 1.92, REG:0.27 | LM NLL: 7.04, KL: 1.82|DE LOSS: 315, PPL: 1293, TM: 308, LM: 6.86|BETA: 0.104092
082[s], Ep: 00, Ct: 02000|TR LOSS: 323, PPL: 1422|TM NLL: 313, KL: 2.16, REG:0.23 | LM NLL: 6.98, KL: 1.63|DE LOSS: 314, PPL: 1266, TM: 307, LM: 6.82|BETA: 0.138766
083[s], Ep: 00, Ct: 02500|TR LOSS: 321, PPL: 1383|TM NLL: 312, KL: 2.33, REG:0.20 | LM NLL: 6.93, KL: 1.47|DE LOSS: 313, PPL: 1228, TM: 306, LM: 6.77|BETA: 0.173440
067[s], 

Nan occured


AttributeError: 'NoneType' object has no attribute 'all_model_checkpoint_paths'

In [28]:
batch[0].idx

46746

In [29]:
for line_idxs in input_token_idxs.eval(session=sess, feed_dict=feed_dict):
    print(' '.join([idx_to_word[idx] for idx in line_idxs]))

# ? <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
# e <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
# e <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
# e <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
# e boston scientific # ? ? ? # ? ? ? # ? ? ? # ? ? ? # ? ? ? # <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <p

# confirm variables

In [26]:
_prob_topic, _prob_topic_sents, _prob_topic_infer, _means_topic_infer = debug_value([prob_topic, prob_topic_sents, prob_topic_infer, means_topic_infer], return_value=True)

In [27]:
batch_i = 4
_prob_topic_sents[batch_i], _prob_topic_infer[batch_i]

(array([1.2281889e-02, 6.7835855e-03, 2.0132679e-01, 1.1674699e-02,
        6.5264981e-03, 1.1411838e-02, 6.9189700e-03, 2.8612482e-04,
        7.4042326e-01, 2.3663496e-03], dtype=float32),
 array([0.09889801, 0.11335436, 0.12141278, 0.10027827, 0.13508864,
        0.09455997, 0.0860846 , 0.08042429, 0.09635323, 0.07354581],
       dtype=float32))

In [28]:
_means_topic_infer[0][:, :4]

array([[-0.02824139, -0.22793637,  0.3788033 , -0.02814816],
       [ 0.06249218, -0.08085692,  0.06756439,  0.2259818 ],
       [-0.1377574 ,  0.28185233,  0.23656711, -0.3201025 ],
       [-0.03711405,  0.16953555,  0.16389738, -0.16461828],
       [ 0.27302447, -0.22462857,  0.37200606, -0.00549014],
       [-0.16755986,  0.11964113,  0.17052333,  0.18083367],
       [ 0.26351342, -0.33828875,  0.3920058 , -0.17029142],
       [ 0.21033517, -0.1955087 ,  0.20963705, -0.24809986],
       [ 0.01922028, -0.03185754,  0.28022933, -0.13928899],
       [-0.26817918, -0.3364467 ,  0.36924574,  0.15818927]],
      dtype=float32)

In [29]:
w_means_topic, b_means_topic = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, "topic/dec/mean_topic")

pred_topic_embeddings, pred_topic_bow, pred_means_topic, pred_logvars_topic, pred_token_idxs, _w_means_topic, _b_means_topic, _w_mean_topic_infer = \
                                sess.run([topic_embeddings, topic_bow, means_topic, logvars_topic, topic_beam_output_token_idxs, w_means_topic, b_means_topic, w_mean_topic_infer], 
                                         feed_dict={t_variables['batch_l']: config.n_topic, t_variables['keep_prob']: 1.,})

pred_sents = idxs_to_sents(pred_token_idxs, config, idx_to_word)

pred_topics_freq_bow_indices = np.argsort(pred_topic_bow, 1)[:, ::-1][:, :10]
pred_topics_freq_bow_idxs = bow_idxs[pred_topics_freq_bow_indices]

In [30]:
for idxs in pred_topics_freq_bow_idxs:
    print([idx_to_word[idx] for idx in idxs])

['million', 'http', 'years', 'tuesday', 'federal', 'u.s.', 'month', 'office', 'people', 'monday']
['reports', 'department', 'thursday', 'court', 'time', 'u.s.', 'years', 'monday', 'tuesday', "'"]
['school', 'public', 'u.s.', 'board', 'university', 'federal', 'president', 'program', 'health', 'students']
['fire', 'found', 'authorities', 'hospital', 'home', 'a.m.', 'dead', 'investigation', 'chief', 'morning']
['service', 'national', 'department', 'http', 'water', 'park', 'area', 'center', 'wednesday', 'weather']
['percent', 'company', 'million', 'billion', 'shares', 'inc.', 'revenue', 'average', 'cents', 'price']
['design', 'researchers', 'donations', 'prize', 'encourage', 'peak', 'awards', 'feature', 'chapter', 'affect']
['court', 'arrested', 'charged', 'attorney', 'death', 'authorities', 'found', 'prison', 'shooting', 'woman']
['bill', 'senate', 'house', 'republican', 'committee', 'public', 'board', 'approved', 'u.s.', 'law']
['todd', 'rear', 'drowned', 'confronted', 'foul', 'transport

In [31]:
pred_topic_embeddings[:, :10]

array([[ 0.09207956,  0.09077395,  0.06120839, -0.22176096,  0.19921999,
        -0.05485373, -0.23069587,  0.05650459,  0.142239  , -0.17410392],
       [ 0.29974467, -0.03281972,  0.25064805, -0.2893645 , -0.1312135 ,
        -0.11136644,  0.02060958, -0.13598587,  0.24592783, -0.18463418],
       [ 0.22819577,  0.14566512,  0.05597903, -0.1109414 , -0.00879419,
        -0.1605561 ,  0.11445126,  0.00916692, -0.04847115, -0.0603605 ],
       [ 0.3365141 , -0.06439152,  0.3631938 ,  0.14203475,  0.10052318,
        -0.09119357, -0.08768138, -0.28562906,  0.3186355 , -0.08452626],
       [ 0.26727033,  0.03540339,  0.28315246, -0.1286456 , -0.02444943,
        -0.163507  , -0.02146725, -0.21729966,  0.26542294, -0.1589102 ],
       [ 0.28826687, -0.18438952,  0.2760168 , -0.12692557,  0.05485712,
         0.17983724,  0.14970489, -0.36168167,  0.30563408, -0.13128197],
       [-0.32988435, -0.11005852, -0.27013943,  0.26166642, -0.3013388 ,
        -0.2234488 ,  0.02287463,  0.20320751

In [32]:
pred_topic_bow

array([[1.0327697e-03, 4.6811462e-03, 1.7024261e-04, ..., 3.3795869e-04,
        1.5871278e-04, 6.6570101e-05],
       [5.9443718e-04, 7.4376534e-03, 2.0111131e-04, ..., 7.4048399e-04,
        1.5028552e-04, 8.9953668e-05],
       [8.5580017e-04, 2.8901210e-03, 1.6464434e-04, ..., 3.1770146e-04,
        1.9349743e-04, 3.0012103e-04],
       ...,
       [1.3188391e-04, 2.6697975e-03, 2.3009101e-04, ..., 8.1575691e-04,
        1.5104198e-04, 3.2140684e-05],
       [3.8558390e-04, 4.0810513e-03, 1.4347673e-04, ..., 1.4157601e-04,
        1.8342295e-04, 4.7484271e-05],
       [7.1679087e-06, 2.9757638e-07, 1.8662996e-04, ..., 5.9841594e-05,
        1.7972868e-04, 5.0798093e-04]], dtype=float32)

In [37]:
_w_means_topic

array([[ 0.09637171, -0.09069463,  0.09532599, ..., -0.02989542,
        -0.02514938,  0.01415634],
       [-0.02764424,  0.01254364,  0.03703775, ..., -0.01512817,
         0.02590305,  0.01404281],
       [-0.08408152, -0.00924051, -0.08365072, ...,  0.05623364,
        -0.05008389,  0.0209127 ],
       ...,
       [-0.00629492, -0.05131948, -0.05206006, ..., -0.0955721 ,
        -0.01980347, -0.02646225],
       [-0.00839109, -0.05062511, -0.05731497, ...,  0.07618995,
         0.03893617,  0.06835916],
       [ 0.07883421, -0.02804748,  0.09326512, ..., -0.0734218 ,
         0.0045918 , -0.10043538]], dtype=float32)

In [36]:
_b_means_topic

array([ 0.03207342, -0.08436021,  0.36105958, -0.02925379, -0.26779142,
       -0.4809294 , -0.04713235, -0.6384421 ,  0.49655244, -0.18943405,
        0.32139724,  0.00376519, -0.02733823,  0.06968141,  0.12723993,
        0.17086434,  0.4321125 ,  0.2930171 ,  0.43750855, -0.32728267,
       -0.29320472,  0.46633536,  0.03467208, -0.11661331,  0.02106199,
        0.10828511,  0.14413024,  0.00801178, -0.16259423,  0.31461757,
       -0.25328413,  0.11320477], dtype=float32)

In [33]:
pred_means_topic

array([[ 0.03563995, -0.3379975 ,  1.6953684 , -0.3394102 , -1.3313038 ,
        -1.9561913 , -0.34439525, -3.1041431 ,  2.302101  , -0.80340064,
         1.374892  ,  0.10505254,  0.13142577,  0.36230466,  0.5036737 ,
         0.6154421 ,  1.8879794 ,  1.263777  ,  2.0281937 , -1.4289141 ,
        -1.3110695 ,  2.0390706 ,  0.32288784, -0.39520237,  0.10320839,
         0.35608286,  0.61754334, -0.1438482 , -0.74413097,  1.4731948 ,
        -1.1249324 ,  0.33894244],
       [ 0.03510073, -0.3374708 ,  1.6888778 , -0.3363807 , -1.3308815 ,
        -1.9608293 , -0.34732953, -3.1081154 ,  2.3005512 , -0.80602133,
         1.3695524 ,  0.10256523,  0.13283278,  0.36182955,  0.49822897,
         0.6213894 ,  1.8883276 ,  1.266403  ,  2.023539  , -1.426166  ,
        -1.3108103 ,  2.0448976 ,  0.32430837, -0.38903922,  0.10560946,
         0.3596735 ,  0.6160856 , -0.14840832, -0.74532497,  1.4738762 ,
        -1.1249926 ,  0.33331037],
       [ 0.03490497, -0.33676392,  1.6840277 , -0.3354

In [40]:
_w_mean_topic_infer[:, :10, 0]

array([[ 0.02438005, -0.02036242, -0.01039507,  0.00277665,  0.02350294,
         0.01473409, -0.02157389,  0.02251465,  0.00814133,  0.01785252],
       [ 0.00074128, -0.001453  , -0.01162554, -0.0014644 , -0.02097618,
         0.00570552,  0.00674119, -0.01879225,  0.02161046,  0.01941887],
       [ 0.01402811, -0.00613734, -0.0027943 , -0.01052572, -0.00041091,
         0.0045091 , -0.01547336,  0.0194314 ,  0.00784631,  0.00505763],
       [ 0.01655601, -0.02403533,  0.01607199, -0.01643238,  0.00686195,
        -0.01788632,  0.01109112,  0.00087933,  0.00448566, -0.00823843],
       [-0.00695892,  0.01631877,  0.01657149,  0.01772625,  0.02026216,
         0.01811663,  0.02153757, -0.00020743, -0.02066918, -0.00025119],
       [ 0.02195978,  0.00867016, -0.02278856,  0.0122384 , -0.02183604,
         0.00798479,  0.00545774,  0.00923075,  0.01814621,  0.02018008],
       [ 0.02242495, -0.00673729,  0.01447002,  0.02163557,  0.02312511,
        -0.00920323, -0.01116639,  0.01708085

In [30]:
_b_means_topic

array([ 0.03922943,  0.46112132, -0.08166478, -0.06917454, -0.01617851,
        0.18225916, -0.28025913, -0.06620797,  0.03231472, -0.02211766,
       -0.38082743,  0.11382294,  0.39363608, -0.4168986 ,  1.1677127 ,
        0.08979508,  0.03593519, -0.07863234,  0.12493958,  0.03175303,
        0.13351525,  0.8176425 , -0.28750482,  0.37927672, -0.01720515,
       -0.21228473, -0.05377329, -0.10451841, -0.24399954, -0.25936088,
        0.0845396 ,  0.33591354], dtype=float32)

In [31]:
_enc_state_infer, _means_topic_infer = debug_value([enc_state_infer, means_topic_infer], return_value=True)

NameError: name 'enc_state_infer' is not defined

In [109]:
_enc_state_infer.shape

(67, 10, 1024)

In [101]:
_means_topic_infer[0]

array([[-4.4230181e-01, -8.2235533e-01, -3.9928532e-01,  3.3004081e-01,
         1.2879696e+00,  2.5491878e-01,  5.9280634e+00,  1.8478313e-01,
         1.1477946e-01,  1.1729380e+00, -3.8528564e+00, -1.1789608e+00,
         9.1431034e-01, -1.7245387e+00, -2.9743382e-01,  1.0960317e+00,
         5.1114732e-01,  8.8052756e-01,  5.0135303e-01,  8.0816686e-01,
        -2.1616230e+00, -1.2552780e+00, -8.6165917e-01, -1.0416836e+00,
         2.4590290e+00, -5.1620197e+00,  1.5814751e-03,  1.3141291e+00,
        -6.7580324e-01,  2.8643382e+00, -2.3818936e+00, -2.5698123e+00],
       [-4.4230181e-01, -8.2235533e-01, -3.9928532e-01,  3.3004081e-01,
         1.2879696e+00,  2.5491878e-01,  5.9280634e+00,  1.8478313e-01,
         1.1477946e-01,  1.1729380e+00, -3.8528564e+00, -1.1789608e+00,
         9.1431034e-01, -1.7245387e+00, -2.9743382e-01,  1.0960317e+00,
         5.1114732e-01,  8.8052756e-01,  5.0135303e-01,  8.0816686e-01,
        -2.1616230e+00, -1.2552780e+00, -8.6165917e-01, -1.0416